In [ ]:
!git clone https://github.com/richzhang/colorization.git

Cloning into 'colorization'...
remote: Enumerating objects: 4873, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 4873 (delta 125), reused 112 (delta 112), pack-reused 4719
Receiving objects: 100% (4873/4873), 177.90 MiB | 28.35 MiB/s, done.
Resolving deltas: 100% (422/422), done.


In [ ]:
cd colorization

/content/colorization


In [ ]:
#!pip install -r requirements.txt

In [ ]:
#pip install scikit-image

In [ ]:
pip install torch scikit-image numpy matplotlib argparse Pillow

In [ ]:
import os
import os
from os.path import isfile, join

In [ ]:
import colorizers
colorizer_eccv16 = colorizers.eccv16().eval()
colorizer_siggraph17 = colorizers.siggraph17().eval()

Downloading: "https://colorizers.s3.us-east-2.amazonaws.com/colorization_release_v2-9b330a0b.pth" to /root/.cache/torch/hub/checkpoints/colorization_release_v2-9b330a0b.pth
100%|██████████| 123M/123M [00:03<00:00, 35.7MB/s]
Downloading: "https://colorizers.s3.us-east-2.amazonaws.com/siggraph17-df00044c.pth" to /root/.cache/torch/hub/checkpoints/siggraph17-df00044c.pth
100%|██████████| 130M/130M [00:03<00:00, 40.1MB/s]


In [ ]:
#!python demo_release.py -i /content/colorization/imgs/images.jpeg -o /content/colorization/imgs_out/output.png

/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 14 pixels
  return func(*args, **kwargs)
Figure(1200x800)


Video

In [ ]:
# import the necessary packages
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os
from os.path import isfile, join

In [ ]:
VIDEO = "rio_32.mp4"

In [ ]:
vs = cv2.VideoCapture(video)

In [ ]:
import cv2
import subprocess

# Open video capture
vs = cv2.VideoCapture('/content/output (2).mp4')  # Replace with the path to your video file

count = 0
success = True

while success:
    success, frame = vs.read()

    if frame is None:
        break

    cv2.imwrite("/content/colorization/imgs/frame%d.jpg" % count, frame)

    # Call the colorization script for each frame
    subprocess.run([
        'python', 'demo_release.py',
        '-i', f'/content/colorization/imgs/frame{count}.jpg',
        '-o', f'/content/colorization/imgs_out/output{count}.png'
    ])

    colorized_frame = cv2.imread(f'/content/colorization/imgs_out/output{count}.png')

    # cv2.imshow("Original", frame)
    # cv2.imshow("Colorized", colorized_frame)

    count += 1

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

vs.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
# def convert_frames_to_video(pathIn, pathOut, fps):
#     frame_array = []
#     files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

#     #for sorting the file names properly
#     files.sort(key = lambda x: int(x[5:-4]))

#     for i in range(len(files)):
#         filename=pathIn + files[i]
#         #reading each files
#         img = cv2.imread(filename)
#         height, width, layers = img.shape
#         size = (width,height)
#         print(filename)
#         #inserting the frames into an image array
#         frame_array.append(img)

#     out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'MJPG'), fps, size)

#     for i in range(len(frame_array)):
#         # writing to a image array
#         out.write(frame_array[i])
#     out.release()

In [ ]:
def convert_frames_to_video(pathIn, pathOut, fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

    # for sorting the file names properly
    files.sort(key=lambda x: int(x[5:-4]) if x[5:-4].isdigit() else 0)

    for i in range(len(files)):
        filename = os.path.join(pathIn, files[i])
        print(f"Reading {filename}")
        img = cv2.imread(filename)

        if img is not None:
            height, width, layers = img.shape
            size = (width, height)
            frame_array.append(img)
        else:
            print(f"Unable to read image: {filename}")

    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    for i in range(len(frame_array)):
        out.write(frame_array[i])

    out.release()


In [ ]:
pathIn= '/content/colorization/imgs'
pathOut = '/content/video.avi'
fps = 30.0
convert_frames_to_video(pathIn, pathOut, fps)

Reading /content/colorization/imgs/.DS_Store
Unable to read image: /content/colorization/imgs/.DS_Store
Reading /content/colorization/imgs/frame0.jpg
Reading /content/colorization/imgs/frame1.jpg
Reading /content/colorization/imgs/frame2.jpg
Reading /content/colorization/imgs/frame3.jpg
Reading /content/colorization/imgs/frame4.jpg
Reading /content/colorization/imgs/frame5.jpg
Reading /content/colorization/imgs/frame6.jpg
Reading /content/colorization/imgs/frame7.jpg
Reading /content/colorization/imgs/frame8.jpg
Reading /content/colorization/imgs/frame9.jpg
Reading /content/colorization/imgs/frame10.jpg
Reading /content/colorization/imgs/frame11.jpg
Reading /content/colorization/imgs/frame12.jpg
Reading /content/colorization/imgs/frame13.jpg
Reading /content/colorization/imgs/frame14.jpg
Reading /content/colorization/imgs/frame15.jpg
Reading /content/colorization/imgs/frame16.jpg
Reading /content/colorization/imgs/frame17.jpg
Reading /content/colorization/imgs/frame18.jpg
Reading /cont

streamlit

In [4]:
!pip install -U gradio

  Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.0
    Uninstalling typing-extensions-3.10.0.0:
      Successfully uninstalled typing-extensions-3.10.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [6]:
#!pip uninstall typing-extensions  # Uninstall the conflicting version
#!pip install typing-extensions==3.10.0.0  # Install the version required by Gradio
!pip install --upgrade gradio  # Install or upgrade Gradio

In [ ]:
import gradio as gr

def get_output(image, user_fun):
    # Implement your image processing logic here based on user_fun
    # For now, just return the original image as a placeholder
    print(image)
    print(user_fun)
    return image

iface = gr.Interface(
    get_output,
    [
        gr.Radio(["Image Restoration", "Image Enhancement", "Image Colourization"], label="Choose a function"),
        gr.Image(label="Upload Image/Video"),  # Use 'sources' to specify upload functionality
    ],
    "image",
    live=True,
)

iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0c0320a34badc9638f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Image Restoration
None


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1570, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1456, in postprocess_data
    outputs_cached = processing_utils.move_files_to_cache(
  File "/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py", line 265, in move_files_to_cache
    return client_utils.traverse(data, _move_to_cache, client_utils.is_file_obj)
  File "/usr/local/lib/python3.10/dist-packages/gradio_client/utils.py", line 919, in traverse
    return func(json_obj)
  File "/usr/local/lib/python3.10